## Facebook Public Group Post And Comment (Generating Response with OpenAI)

In [ ]:
!pip install openai

In [36]:
import numpy as np
import pandas as pd
import time
import openai
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.indexes import VectorstoreIndexCreator
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI

In [40]:
dataframe = pd.read_csv("../../Sample Datasets/classify_sentiment_smsa_data.csv")
pd.set_option('display.max_colwidth', None)
dataframe.head(3)

,Post,1Comment,2Comment,tokens,sentiment,textblob_polarity
0,i feel like i m a burden to everybody bothering people with my problems my existence moving to a different city changing my number have been heavy on my mind,best to talk to a therapist it s too much for family and friends,i hear you i do i want to do the same sometimes but then i watch people and see who they are because i know i have a mental issue and i know my brain lies to me look around you some people really do care maybe go to weekly mind meetings in the uk or groups local to you who all feel the same xxx,"['i', 'feel', 'like', 'i', 'm', 'a', 'burden', 'to', 'everybody', 'bothering', 'people', 'with', 'my', 'problems', 'my', 'existence', 'moving', 'to', 'a', 'different', 'city', 'changing', 'my', 'number', 'have', 'been', 'heavy', 'on', 'my', 'mind']",Moderately Negative,-0.1
1,this is what i ll be doing on christmas i just do not feel well i just do not have it in me to fight i ve been doing it for years i m just worn out i do hope your christmas will be better,give yourself a break from your negative thoughts this is the day the king is born,same here i know how you feel,"['this', 'is', 'what', 'i', 'll', 'be', 'doing', 'on', 'christmas', 'i', 'just', 'do', 'not', 'feel', 'well', 'i', 'just', 'do', 'not', 'have', 'it', 'in', 'me', 'to', 'fight', 'i', 've', 'been', 'doing', 'it', 'for', 'years', 'i', 'm', 'just', 'worn', 'out', 'i', 'do', 'hope', 'your', 'christmas', 'will', 'be', 'better']",Positive,0.5
2,bye big hugs blessed be judge me all you want don t care after tonight my spouse thinks he s so wonderful well let s me make it real,i m okay but mentally i m not and i m done putting up with all this i will be okay when i disappear not dead just gone i ve seriously had enough thank you all for caring i do greatly appreciate it more than i can even express,are you ok,"['bye', 'big', 'hugs', 'blessed', 'be', 'judge', 'me', 'all', 'you', 'want', 'don', 't', 'care', 'after', 'tonight', 'my', 'spouse', 'thinks', 'he', 's', 'so', 'wonderful', 'well', 'let', 's', 'me', 'make', 'it', 'real']",Moderately Positive,0.4


In [41]:
sam_df = dataframe
sam_df.drop(columns = ['1Comment', '2Comment', 'tokens', 'sentiment', 'textblob_polarity'], inplace = True)
#sample subset of data
sam_df = sam_df.sample(n = 500, random_state = 42)
sam_df.head(3)

,Post
2898,what is wrong with people that sexually abuse others
2976,just a reminder of the causes we are supporting our primary aim is to help remove the stigma associated with mental health and talking about it nobody should ever feel alone and isolated we feel that fundraising for mindfixers socially inclusive therapy beachy head chaplaincy team suicide prevention and pavilion drug and alcohol support is the best place to start mindfixers is a not for profit organisation created to provide a cheap quick and top quality solution for those seeking therapy part of the mindfixers solution mindfixers programme is a hour programme spread over weeks which is accessible by anyone and provides therapy for almost any mental condition the cost of an hour s session is around less than the average hourly rate for private therapy and can often be free of charge accessibility is the key and mindfixers will enable thousands of people who would otherwise be denied therapy the ability to fix their mental issues fast quickly and expertly beachy head chaplaincy is a valued search and rescue charity that seeks to save lives at beachy head east sussex the chaplains patrol on foot and by car and respond to emergency calls locating anyone at risk using their skills in crisis intervention they offer supportive listening to start a dialogue and to encourage more hopeful solutions than suicide they work alongside local services helping people to access support pavillions provides drug and alcohol services for adults in brighton hove support is available to anyone concerned about their drug or alcohol use or to the families and carers supporting those struggling with substance misuse irrespective of age gender disability ethnicity sexual orientation marital status or hiv status as you can see these are amazing causes please donate
2130,well it happened after the latest health news and a big argument with my wife being sick of health problems while saying she wants to stay friends said i m not capable of being a husband while having all these health problems and unless i can find a treatment that works she wants a separation as what we have is not even a marriage anymore it s just us moaning about our lives to each other with having autism and how badly i am at socalising that is no doubt my one and only chance at love destroyed


In [43]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxx"

doclist = []

for eachpost in range(sam_df.shape[0]):
    raw_documents = Document(page_content = "sm_posts")
    doclist.append(raw_documents)
    
print("Loading",len(doclist),"social media posts into the VectorstoreIndex as examples.")

Loading 500 social media posts into the VectorstoreIndex as examples.


In [44]:
index = VectorstoreIndexCreator(vectorstore_kwargs = {"persist_directory":None}).from_documents(doclist)
index

VectorStoreIndexWrapper(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001AFE1CF1750>)

In [45]:
query1 = "Use Social Media Posts that I provided: " + \
str(raw_documents) + \
"From the perspective of the National Institute of Mental Health " + \
"as a psychologist, " + \
"Examine these posts and assess each if it contains any mental health disorder " + \
"Pay attention to the language used, any expressions of depression, anxiety, stress, Post-traumatic stress disorder (PTSD), feelings of being trapped or overwhelmed, or mentions of wanting to end ones' life " + \
"Also, look for any indications of hopelessness, helplessness, hate, being tired, or worthlessness, as well as any mention of plans or preparations for self-harm. " + \
"Take every post seriously." + \
"Use a diagnosis of Follow-up with any potential things checked for above and give some advice or recommendation for mental health." + \
"You must answer as a python dictionary with keys post_number and diagnosis is in the format " + \
"{ post: [{post_number: x, diagnosis: value}]}."

query2 = "Retry the posts " + \
str(raw_documents) + \
"From the perspective of the National Institute of Mental Health " + \
"as a counsellor, " + \
"Examine these posts and assess each if it contains any mental health disorder " + \
"Pay attention to the language used, any expressions of depression, anxiety, stress, Post-traumatic stress disorder (PTSD), feelings of being trapped or overwhelmed, or mentions of wanting to end ones' life " + \
"Also, look for any indications of hopelessness, helplessness, hate, being tired, or worthlessness, as well as any mention of plans or preparations for self-harm. " + \
"Take every post seriously." + \
"Use a diagnosis of Follow-up with any potential things checked for above and give some advice or recommendation for mental health." + \
"You must answer as a python dictionary with keys post_number and diagnosis is in the format " + \
"{ post: [{post_number: x, diagnosis: value}]}."

In [ ]:
BATCH_SIZE = 1
#sam_df.shape[0]
batch_cnt = 20 
response = ""
responsedict = []

for batch in range(batch_cnt):
    raw_documents = Document(page_content = str(sam_df.loc[batch*BATCH_SIZE:(batch+1)*BATCH_SIZE-1, "Post"].to_dict()))
    print("Processing batch",batch+1,"of",batch_cnt,"batches.")
    
    query1 = "Use Social Media Posts that I provided: " + \
    str(raw_documents) + \
    "From the perspective of the National Institute of Mental Health " + \
    "as a psychologist, " + \
    "Make a diagnosis and response when a post has mental health disorder " + \
    "such as depression, anxiety, stress, Post-traumatic stress disorder (PTSD), feelings of being trapped or overwhelmed, or mentions of wanting to end ones' life. " + \
    "And also categorize the mental health disorder, properly." + \
    "It is better have a diagnosis as Follow-up than a diagnosis as Ignore and double check before a diagnosis for Ignore." + \
    "You must answer as a python dictionary with keys post_number and diagnosis is in the format " + \
    "{ post: [{post_number: x, diagnosis: value}]}."
     
    query2 = "Retry the posts" + \
    str(raw_documents) + \
    "From the perspective of the National Institute of Mental Health " + \
    "as a counsellor, " + \
    "Make a diagnosis and response when a post has mental health disorder " + \
    "such as depression, anxiety, stress, Post-traumatic stress disorder (PTSD), feelings of being trapped or overwhelmed, or mentions of wanting to end ones' life. " + \
    "And also categorize the mental health disorder, properly." + \
    "It is better have a diagnosis as Follow-up than a diagnosis as Ignore and double check before a diagnosis for Ignore." + \
    "You must answer as a python dictionary with keys post_number and diagnosis is in the format " + \
    "{ post: [{post_number: x, diagnosis: value}]}."   
    
    try:
        response = index.query(query1, llm = ChatOpenAI(model_name = "gpt-3.5-turbo"))
    except:
        response = index.query(query2, llm = ChatOpenAI(model_name = "gpt-3.5-turbo"))
        
    responsedict.append(response)
    time.sleep(1)
    
print("done")
responsedict[:10]